In [242]:
import numpy as np
from pprint import pprint as pp
import re
import math as maths
from scipy.optimize import minimize
from scipy.optimize import Bounds
from ase.units import kB

pattern1 = re.compile("\n\n\n")
pattern2 = re.compile("\n\n")

In [2]:
# Read clusters.out
clusters = {}


with open('clusters.out','r') as fclusters:
    temp_clusters = fclusters.read().split('\n\n')

for idx, cluster in enumerate(temp_clusters):
    if cluster == '':
        continue
    line = cluster.split('\n')
    multiplicity = int(line[0])
    length = float(line[1])
    num_points = int(line[2])
    clusters[idx] = {'mult':multiplicity, 'length':length, 'type':num_points}
    
num_clusters = len(clusters)

In [3]:
# Read config.out
configs = {}

fconfig = open('config.out','r')
_ = next(fconfig)
temp_config = fconfig.read()#.split('\n\n')
temp_config = pattern1.split(temp_config)

for idx, config in enumerate(temp_config):
    if config == '':
        continue
    num_points = int(config[0])
    config = pattern2.split(config[2:])
    min_coords = []
    for _ in range(num_points):
        min_coords.append(config[_].split('\n')[0])
    configs[idx] = {'subclus': list(map(int,min_coords)), 'num_of_subclus': len(min_coords)}

In [4]:
# Read kb.out

kb = {}

fkb = open('kb.out','r')
_ = next(fkb)

temp_kb = fkb.read()
temp_kb = temp_kb.split('\n')

for idx, kbcoeff in enumerate(temp_kb):
    if kbcoeff == '':
        continue
    kb[idx] = float(kbcoeff)

fkb.close()

In [5]:
# Read configcoeff.out

configcoef = {}

with open('configcoef.out','r') as fsubmult:
    _ = next(fsubmult)
    temp_submult = fsubmult.read()
    temp_submult = pattern2.split(temp_submult)
    
for idx, submult in enumerate(temp_submult):
    submult = submult.split('\n')
    while("" in submult) :
        submult.remove("")
    configcoef[idx] = list(map(float,submult[1:]))

In [6]:
# Read vmat.out

vmat = {}

with open('vmat.out') as fvmat:
    _ = next(fvmat)
    temp_vmat = fvmat.read()
    #temp_vmat = temp_vmat.split('\n')
    temp_vmat = pattern2.split(temp_vmat)
    
    while("" in temp_vmat) :
        temp_vmat.remove("")
    
    for clus_idx, mat in enumerate(temp_vmat):
        mat = mat.split('\n')
        mat_float = np.empty(list(map(int, mat[0].split(' '))))
        for idx, row in enumerate(mat[1:]):
            mat_float[idx] = list(map(float,row.split(' ')[:-1]))
        
        vmat[clus_idx] = mat_float

In [173]:
# Read eci
eci = {}

with open('eci.out') as feci:
    _ = next(feci)
    temp_eci = feci.read()
    temp_eci = temp_eci.split('\n')

for idx, eci_val in enumerate(temp_eci):
    if eci_val == '':
        continue
    eci[idx] = float(eci_val)

eci

{0: 0.006143, 1: -0.018951, 2: 0.01147, 3: -0.009881, 4: 2.3e-05, 5: 0.002117}

In [136]:
corrs = np.array([1.0,-0.14286,0.02041,0.02041,0.00292,0.0])

## Doing the Optimisation

In [456]:
corrs = np.array([1.0,0.2,0.02041,0.02041,0.00292,0.0])

#cluster_optim = []
#for cluster_idx, cluster in clusters.items():
#    cluster_optim.append(cluster['type'] >= 2)
    
#corrs_optim = corrs[cluster_optim]
#corrs_fixed = corrs[[not elem for elem in cluster_optim]]
FIXED_CORR_1 = 0.2

bounds_corrs = Bounds(np.array([-1]*len(corrs)),np.array([1]*len(corrs)))

def constraint_rhos(corrs, vmat, clusters, configcoef,):
    
    all_pos = []
    rho_sum = []
    
    #corrs = np.concatenate((corrs_fixed,corrs_optim))
    def clus_prob(cluster_idx):
        rho = np.matmul(vmat[cluster_idx],corrs)
        return rho
    
    for cluster_idx, _ in clusters.items():
        rho = clus_prob(cluster_idx)
        all_pos.append(((rho >= 0).all() and (rho <= 1).all()))
        rho_sum.append(np.sum(configcoef[cluster_idx]*rho))
    
    return np.sum(1 - np.array(rho_sum)) + (1 - int(all(all_pos)))

def constraint_singlet(corrs):
    
    return corrs[1] - FIXED_CORR_1
                    
       
def calc_S(corrs, vmat, kb, clusters, configs, configcoef,):

    S = 0
    H = 0
    #corrs = np.concatenate((corrs_fixed,corrs_optim))
    
    def clus_prob(cluster_idx):
        rho = np.matmul(vmat[cluster_idx],corrs)

        return rho
    
    def inner_sum(cluster_idx):
        isum = 0
        rho = clus_prob(cluster_idx)
        #print(rho)
        try:
            for i in range(configs[cluster_idx]['num_of_subclus']):
                isum += configcoef[cluster_idx][i] * rho[i] * maths.log(rho[i])
        except ValueError as ve:
            #print(ve)
            pass
        #for i in range(configs[cluster_idx]['num_of_subclus']):
        #        isum += configcoef[cluster_idx][i] * rho[i] * np.log(rho[i])
        return isum 
            
    for cluster_idx, cluster in clusters.items():
        H += cluster['mult']*eci[cluster_idx]*corrs[cluster_idx]
        S += kb[cluster_idx]*inner_sum(cluster_idx)

    T = 3000
    
    #print(corrs)
    return S
    #return H - T*kB*S

In [457]:
res = minimize(calc_S, 
               corrs, 
               args=(vmat, kb, clusters, configs, configcoef,),
               method='trust-constr', 
               options={'verbose': 3,},#'gtol': 1e-3, 'maxiter': 200},
               jac='3-point',
               constraints=[{"fun": constraint_rhos, "type": "eq", 'args': (vmat, clusters, configcoef,)},
                            {"fun": constraint_singlet, "type": "eq"}
                           ],
               bounds=bounds_corrs
              )

| niter |f evals|CG iter|  obj func   |tr radius |   opt    |  c viol  | penalty  |barrier param|CG stop|
|-------|-------|-------|-------------|----------|----------|----------|----------|-------------|-------|
|   1   |  13   |   0   | -2.6909e+00 | 1.00e+00 | 4.12e-02 | 2.22e-16 | 1.00e+00 |  1.00e-01   |   0   |
|   2   |  26   |   1   | -2.6700e+00 | 5.61e+00 | 2.44e-02 | 2.48e-02 | 1.00e+00 |  1.00e-01   |   4   |
|   3   |  39   |   2   | -2.6815e+00 | 5.61e+00 | 5.55e-03 | 1.24e-02 | 1.00e+00 |  1.00e-01   |   4   |
|   4   |  52   |   3   | -2.6869e+00 | 5.61e+00 | 2.16e-03 | 6.25e-03 | 1.00e+00 |  1.00e-01   |   4   |
|   5   |  65   |   4   | -2.6895e+00 | 5.61e+00 | 6.71e-04 | 3.14e-03 | 1.00e+00 |  1.00e-01   |   4   |
|   6   |  65   |   4   | -2.6895e+00 | 2.80e+01 | 2.66e-03 | 3.14e-03 | 1.00e+00 |  2.00e-02   |   0   |
|   7   |  78   |   5   | -2.6908e+00 | 2.80e+01 | 1.93e-03 | 1.58e-03 | 1.00e+00 |  2.00e-02   |   4   |
|   8   |  91   |   7   | -2.6914e+00 | 2.80e+

/home/sayan/.local/lib/python3.8/site-packages/scipy/optimize/_hessian_update_strategy.py:182: UserWarning: delta_grad == 0.0. Check if the approximated function is linear. If the function is linear better results can be obtained by defining the Hessian as zero instead of using quasi-Newton approximations.
  warn('delta_grad == 0.0. Check if the approximated '


|  46   |  468  |  60   | -2.6920e+00 | 5.47e-01 | 4.11e-11 | 1.11e-08 | 1.00e+00 |  1.02e-08   |   4   |
|  47   |  481  |  62   | -2.6920e+00 | 2.74e-01 | 4.11e-11 | 1.11e-08 | 1.00e+00 |  1.02e-08   |   4   |
|  48   |  494  |  66   | -2.6920e+00 | 2.74e-01 | 1.68e-09 | 1.11e-08 | 1.00e+00 |  1.02e-08   |   1   |
|  49   |  507  |  68   | -2.6920e+00 | 1.37e-01 | 1.68e-09 | 1.11e-08 | 1.00e+00 |  1.02e-08   |   4   |
|  50   |  520  |  70   | -2.6920e+00 | 6.84e-02 | 1.68e-09 | 1.11e-08 | 1.00e+00 |  1.02e-08   |   4   |
|  51   |  533  |  72   | -2.6920e+00 | 3.42e-02 | 1.68e-09 | 1.11e-08 | 1.00e+00 |  1.02e-08   |   4   |
|  52   |  546  |  74   | -2.6920e+00 | 1.71e-02 | 1.68e-09 | 1.11e-08 | 1.00e+00 |  1.02e-08   |   4   |
|  53   |  559  |  76   | -2.6920e+00 | 8.55e-03 | 1.68e-09 | 1.11e-08 | 1.00e+00 |  1.02e-08   |   4   |
|  54   |  572  |  79   | -2.6920e+00 | 4.28e-03 | 1.68e-09 | 1.11e-08 | 1.00e+00 |  1.02e-08   |   4   |
|  55   |  585  |  81   | -2.6920e+00 | 2.14e-

In [458]:
res

 barrier_parameter: 2.048000000000001e-09
 barrier_tolerance: 2.048000000000001e-09
          cg_niter: 204
      cg_stop_cond: 4
            constr: [array([1.10653059e-08]), array([7.77159448e-11]), array([1.    , 0.2   , 0.04  , 0.04  , 0.008 , 0.0016])]
       constr_nfev: [658, 658, 0]
       constr_nhev: [0, 0, 0]
       constr_njev: [0, 0, 0]
    constr_penalty: 1.0
  constr_violation: 1.106530589378707e-08
    execution_time: 0.43779945373535156
               fun: -2.692046664554395
              grad: array([-1.85423271e+00,  8.10930218e-01,  4.40022344e-10, -1.46674115e-10,
        2.20011172e-10,  2.93348229e-10])
               jac: [array([[67108858.00000001,        0.        ,        0.        ,
               0.        ,        0.        ,        0.        ]]), array([[0., 1., 0., 0., 0., 0.]]), array([[1., 0., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0.],
       [0., 0., 1., 0., 0., 0.],
       [0., 0., 0., 1., 0., 0.],
       [0., 0., 0., 0., 1., 0.],
       [0., 

In [459]:
for cluster_idx in clusters:
    print(np.matmul(vmat[cluster_idx],res.x))

[1.]
[0.4 0.6]
[0.16 0.24 0.36]
[0.16 0.24 0.36]
[0.064 0.096 0.144 0.096 0.144 0.216]
[0.0256 0.0384 0.0576 0.0576 0.0864 0.1296]
